# Orbit Calibration Set Up  
This notebook will download the relevant data from space-track from the sp3 files that you provide. This will be then stored in the rawdata folder that can then be used for the calibration analysis. 

In [45]:
import os
import sp3
from datetime import datetime, timedelta
from dotenv import load_dotenv
import requests
import json
import spacetrack.operators as op
from spacetrack import SpaceTrackClient
import pickle

st = SpaceTrackClient(identity='indigo.brownhall.20@ucl.ac.uk', password='Greeningplant123!')

In [37]:
# manually fill out the folder where your s3 data for grace-fo-1 and grace-fo-2 is stored
RawDataFolderName = 'v1'
fileType = "sp3"

# get the current working directory
cwd = os.getcwd()
RawDataFolder = os.path.join(cwd, 'RawData', RawDataFolderName)

# get all the files in the RawDataFolder
files = os.listdir(RawDataFolder)

if fileType == "pickle":
    sp3files = [file for file in files if file.endswith('.pkl')]
    grace1Name = [file for file in sp3files if 'L64' in file]
    grace2Name = [file for file in sp3files if 'L65' in file]
    with open(os.path.join(os.getcwd(), 'RawData', RawDataFolder, grace1Name[0]), 'rb') as file:
        grace1 = pickle.load(file)
    # same for grace2
    with open(os.path.join(os.getcwd(), 'RawData', RawDataFolder, grace2Name[0]), 'rb') as file:
        grace2 = pickle.load(file)

    start_date = min(grace1[0].time, grace2[0].time) - timedelta(days=2)
    end_date = min(grace1[-1].time, grace2[-1].time) + timedelta(days=2)

    print('Start date: {}'.format(start_date))
    print('End date: {}'.format(end_date))

if fileType == 'sp3':
    sp3files = [file for file in files if file.endswith('.sp3')]
    grace1Name = [file for file in sp3files if 'L64' in file]
    grace2Name = [file for file in sp3files if 'L65' in file]
    grace1 = sp3.Product.from_file(os.path.join(os.getcwd(), 'RawData', RawDataFolder, grace1Name[0]))
    grace2 = sp3.Product.from_file(os.path.join(os.getcwd(), 'RawData', RawDataFolder, grace2Name[0]))

    # get the start and date
    start_date = min(grace1.satellites[0].records[0].time, grace2.satellites[0].records[0].time) - timedelta(days=2)
    end_date = max(grace1.satellites[0].records[-1].time, grace2.satellites[0].records[-1].time) + timedelta(days=2)

    print('Start date: {}'.format(start_date))
    print('End date: {}'.format(end_date))

Start date: 2023-09-22 09:59:42+00:00
End date: 2023-09-27 00:00:12+00:00


In [43]:
start_date.strftime('%Y-%m-%d %H:%M:%S')
epoch=f">{start_date.strftime('%Y-%m-%d %H:%M:%S')}&<{end_date.strftime('%Y-%m-%d %H:%M:%S')}"
epoch

'>2023-09-22 09:59:42&<2023-09-27 00:00:12'

In [53]:
# need to update the date to take out the time to be dynamic
grace1_tle = st.tle(norad_cat_id=43476, epoch=f">{start_date.strftime('%Y-%m-%d %H:%M:%S')}&<{end_date.strftime('%Y-%m-%d %H:%M:%S')}",orderby='epoch asc', format='json')
grace2_tle = st.tle(norad_cat_id=43477, epoch=f">{start_date.strftime('%Y-%m-%d %H:%M:%S')}&<{end_date.strftime('%Y-%m-%d %H:%M:%S')}",orderby='epoch asc', format='json')

# save in the RawData folder with a name of grace-fo-1-tle-43476.json
with open(os.path.join(RawDataFolder, 'grace-fo-1-tle-43476.json'), 'w') as f:
    f.write(grace1_tle)

with open(os.path.join(RawDataFolder, 'grace-fo-2-tle-43477.json'), 'w') as f:
    f.write(grace2_tle)

ReadTimeout: The read operation timed out